In [1]:
from selenium.webdriver import Chrome, ChromeOptions
from pyvirtualdisplay import Display
import pandas as pd
from time import sleep
from util import get_engine, get_upload_url
from urllib.parse import urlparse
from random import randint, shuffle, choice
from time import time
import requests
from shutil import make_archive

In [2]:
cat = 'Arts and Entertainment'
profile_path = f'profiles/{cat}'

In [3]:
# Display(size=(1920,1080)).start()

options = ChromeOptions()
options.add_argument('--window-size=1920,1080')
options.add_argument('--no-sandbox')
options.add_argument(f'--user-data-dir={profile_path}')

driver = Chrome(options=options)
driver.implicitly_wait(30)

In [ ]:
def get_urls(category):
    return pd.read_sql(f'SELECT URL FROM similarweb WHERE Category = "{category}" ORDER BY "Rank"', con=get_engine())['URL']

In [ ]:
def get_random_text():
    with open('topics.txt') as f:
        texts = f.read().split('\n')
        return choice(texts).strip()                

def loiter(driver, how_long=300):
    start_time = time()
    
    # spend the required time around the site
    while (time() - start_time) < how_long:
        
        try:
            action = randint(0, 5)
            if action == 0:
                # execute a random scroll
                driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight * Math.random())", "");
            elif action == 1:
                # get domain name
                dom = urlparse(driver.current_url).netloc
                # get all `a` links
                linkEls = driver.find_elements_by_tag_name('a')
                # shuffle links so random is clicked
                shuffle(linkEls)

                for link in linkEls[:5]:
                    href = link.get_attribute('href')
                    # for each link, check for non-empty href to same domain
                    if href is not None and href.strip() != '' and dom == urlparse(href).netloc:
                        # click the link
                        try:
                            link.click()
                            break
                        except:
                            pass
            elif action == 2:
                # get all `input` fields
                inputEls = driver.find_elements_by_tag_name('input')
                # shuffle fields so random is selected
                shuffle(inputEls)

                text = get_random_text()
                for inputt in inputEls[:5]:
                    try:
                        inputt.send_keys(text + '\n')
                        break
                    except:
                        pass
            else:
                pass
        except:
            pass

In [ ]:
for url in get_urls(cat)[:5]:
    driver.get(f'https://{url}')
    loiter(driver, how_long=30)

In [ ]:
driver.close()

In [55]:
from datetime import datetime
import os
os.path.basename(archive)

'Arts and Entertainment.zip'

In [72]:
def get_filename(archive, timestamp):
    basename = os.path.basename(archive)
    name, ext = os.path.splitext(basename)
     ts = datetime.strftime(timestamp, '%Y%m%d%H%M%S')
    return f'{name}-{ts}{ext}'

In [27]:
archive = make_archive(cat, 'zip', profile_path, '.')

In [39]:
url = get_upload_url()
files = {'file': open(archive, 'rb')}
name = get_filename(archive, timestamp)
r = requests.post(url,data={'key': 'froam_haroon', 'filename': name}, files=files)

In [73]:
get_filename(archive, datetime.now())

'Arts and Entertainment-20210511211337.zip'

In [67]:
datetime.strftime(datetime.now(), '%Y%m%d%H%M%S')

'20210511211110'

In [ ]:
pd.DataFrame([{
    'Category': cat,
    'Time': datetime.now(),
    'Filename': os.path.basename(archive)
}]).to_sql('crawls')